In [24]:
# Sam Brown
# sam_brown@mines.edu
# June 19
# Goal: Build neural net to classify "impulsivitiy" of slip

import sys
sys.path.append("/Users/sambrown04/Documents/SURF/whillans-surf/notebooks/SURF")

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


df = pd.read_csv('averages_events_2011-13')
df = df.iloc[1:-1] # account for shift to get mins until feature

#Endode out target
df['impulsive'] = (df['slip_severity'] > 9* (10 ** -7)).astype(int)

In [28]:
# Define Features and Target
X = df[['tide_height', 'tide_change', 'mins_since', 'h_event', 'form_factor']]
y = df['impulsive'].values.reshape(-1, 1)  # Keep as 0/1

# Split to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42)

# Standardize only features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [30]:
# Neural net
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(5,20)
        self.fc2 = nn.Linear(20,10)
        self.fc3 = nn.Linear(10,1)

    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x)) # sigmoid for classification
        return x



In [32]:
# Instantiate
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = .01)

In [34]:
# Training loop
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad() # Clears grad

    # Predictions and loss
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backprop
    loss.backward()

    # Update params
    optimizer.step()
    
    if (epoch+1) % 20 == 0: 
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [20/200], Loss: 0.1210
Epoch [40/200], Loss: 0.0899
Epoch [60/200], Loss: 0.0844
Epoch [80/200], Loss: 0.0804
Epoch [100/200], Loss: 0.0770
Epoch [120/200], Loss: 0.0748
Epoch [140/200], Loss: 0.0729
Epoch [160/200], Loss: 0.0710
Epoch [180/200], Loss: 0.0693
Epoch [200/200], Loss: 0.0678


In [36]:
model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_tensor)  # probabilities from sigmoid
    y_pred_labels = (y_pred_probs >= 0.5).int()  # convert to 0 or 1

# Convert tensors to numpy arrays for metrics
y_pred_labels_np = y_pred_labels.numpy().flatten()
y_test_np = y_test_tensor.numpy().flatten()

# Print evaluation results
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_np, y_pred_labels_np))

print("\nClassification Report:")
print(classification_report(y_test_np, y_pred_labels_np))

print("\nAccuracy:", accuracy_score(y_test_np, y_pred_labels_np))


Confusion Matrix:
[[158  16]
 [ 16 139]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91       174
         1.0       0.90      0.90      0.90       155

    accuracy                           0.90       329
   macro avg       0.90      0.90      0.90       329
weighted avg       0.90      0.90      0.90       329


Accuracy: 0.9027355623100304
